In [3]:
import os
import itertools
import json
import numpy as np

# Constants
POLLUTANTS = {'CF4': '1', 'CHF3': '2', 'C2F6': '3'}
PORTS = {'port1': '1', 'port4': '4', 'port6': '6', 'port8': '8'}
firing_conditions_mapping = {
    '27.5kW' : ('275', 0.0180445578, 0.0004855154),
    '34kW': ('34', 0.018199937, 0.000611719),
    '41kW': ('32', 0.017959938, 0.000740736),
    '45kW': ('45',  0.017999938, 0.000811917),
}
INTERNAL_FOLDERS = ['dot', 'excel', 'reaction_paths']
VALID_FILE_EXTENSIONS = ['.py', '.ipynb', '.txt', '.csv']

# Creating .gitignore content
gitignore_content = "*\n!*.py\n!*.ipynb\n!*.txt\n!*.csv"

# Select conditions - for your use case, you may adjust these lists
selected_pollutants = ['CF4', 'CHF3', 'C2F6']
selected_ports = ['port1', 'port4', 'port6', 'port8']
selected_firing_conditions = ['27.5kW', '45kW']
selected_streamlines = [16]

main_dir = os.path.join(os.getenv('USERPROFILE'), 'Desktop', 'PFAS_Modeling')
net_dir = os.path.join(main_dir, 'cfs_runs', '0000_docs')
template_file_path = os.path.join(net_dir, '000_rainbow.netj') #path 
# Specify base directory - change to your preferred directory
base_directory = os.path.join(main_dir, 'cfs_runs')

# Create combinations of conditions
for pollutant, port, firing_condition, streamline in itertools.product(selected_pollutants, selected_ports, selected_firing_conditions, selected_streamlines):
    
    run_number = 0 # Initialize run_number
    base_folder_name = f"{POLLUTANTS[pollutant]}{PORTS[port]}.{firing_conditions_mapping[firing_condition][0]}.{streamline}"
    
    # Check if folder exists and adjust run_number accordingly
    while os.path.exists(os.path.join(base_directory, f"{base_folder_name}.{run_number}")):
        run_number += 1
    
    # Create folder name
    if run_number != 0:
        folder_name = f"{base_folder_name}.{run_number}_{firing_condition}_{pollutant}_{port}_{streamline}stream"
    else:
        folder_name = f"{base_folder_name}_{firing_condition}_{pollutant}_{port}_{streamline}stream"
    # Create folder and internal folders
    os.makedirs(os.path.join(base_directory, folder_name), exist_ok=True)
    for internal_folder in INTERNAL_FOLDERS:
        os.makedirs(os.path.join(base_directory, folder_name, internal_folder), exist_ok=True)
    
    # Create .gitignore
    with open(os.path.join(base_directory, folder_name, ".gitignore"), "w") as file:
        file.write(gitignore_content)
    
    # Create null.txt
    with open(os.path.join(base_directory, folder_name, "null.txt"), "w") as file:
        file.write("asdf")

    work_dir_folder = f'{base_folder_name}_{firing_condition}_{pollutant}_{port}_{streamline}stream'
    work_dir_path = os.path.join(base_directory, work_dir_folder)

    with open(template_file_path, 'r') as template_file:
        template_data = json.load(template_file)

        _, airM_val, gasM_val = firing_conditions_mapping[firing_condition]
        template_data['injection_port'] = int(PORTS[port])
        template_data['airM'][0] = airM_val # Replace the first value in airM
        template_data['gasM'][0] = gasM_val # Replace the first value in gasM
        template_data['work_dir']= work_dir_path # Replace work_dir with the new path
        template_data['num_streamlines'] = np.sqrt(streamline) # Set the number of streamlines
        template_data['state'][0] = 0
        template_data['state'][1] = 0
        template_data['state'][2] = 0

        # Set all species concentrations to 0.0 except for the pollutant
    for species in template_data['species_concentrations']:
        if species == pollutant:
            template_data['species_concentrations'][species] = 100.0
        else:
            template_data['species_concentrations'][species] = 0.0

        if pollutant == 'C2F6':
            template_data['species_concentrations']['CF3CF3'] = 100.0

    output_file_path = os.path.join(net_dir, f'{work_dir_folder}.netj')
    with open(output_file_path, 'w') as output_file:
        json.dump(template_data, output_file, indent=4) # Write JSON with proper formatting
